In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/images_preprocessed.zip .

In [ ]:
!unzip images_preprocessed.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout,ZeroPadding2D, BatchNormalization
from tensorflow.keras import regularizers

In [ ]:
#Settings
num_batches = 31
learning_rate = 0.0001
num_epochs = 100
width = 128
height = 128
seed = 2021
encoding_size = 100

In [ ]:
#Load images
train_dataset = keras.preprocessing.image_dataset_from_directory(
    "images_preprocessed",
    label_mode = None,
    color_mode = "rgb",
    batch_size = num_batches,
    image_size = (width, height),
    seed = seed
)

Found 11470 files belonging to 1 classes.


In [ ]:
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
#Create neural network structure
generator = keras.models.Sequential([
    keras.layers.Dense(32 * 32 * 128, input_shape=[encoding_size]),
    keras.layers.Reshape([32, 32, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="SAME", activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(3, kernel_size=5, strides=2, padding="SAME", activation="tanh"),
])

#Evaluating images
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="SAME", activation=keras.layers.LeakyReLU(0.2), input_shape=[width, height, 3]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="SAME", activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")
])

gan_model = keras.models.Sequential([generator, discriminator])

In [ ]:
#Compile
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan_model.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
#Change data to range -1 to 1 because of tanh activation in generator
train_dataset = train_dataset.map(lambda x: (x / 255.) * 2. - 1.)

In [ ]:
%matplotlib inline
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    images = ((images + 1) / 2.)
    plt.figure(figsize=(n_cols*2, n_rows*2))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

In [ ]:
#Custom training
for epoch in range(num_epochs):
    print(epoch)
    for actual_batch in train_dataset:
        #Train discriminator (Image critic)
        noise = tf.random.normal(shape=[num_batches, encoding_size]) #Generate gaussian noise 
        generated_images = generator(noise) #Create images from noise
        generated_and_real = tf.concat([generated_images, actual_batch], axis=0) #Concatenate real and fake images 
        labels_for_discriminator = tf.constant([[0.]] * num_batches + [[1.]] * num_batches) #Label images, 0 for fake 1 for real images
        discriminator.trainable = True
        discriminator.train_on_batch(generated_and_real, labels_for_discriminator)
        
        #Train generator (Image painter)
        noise = tf.random.normal(shape=[num_batches, encoding_size])
        labels_for_generator = tf.constant([[1.]] * num_batches)
        discriminator.trainable = False
        gan_model.train_on_batch(noise, labels_for_generator)
    plot_multiple_images(generated_images, 10)
    plt.show()